<a href="https://colab.research.google.com/github/Axhay04/Axhay04/blob/main/Reviewers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade gspread gspread-dataframe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.2.0
    Uninstalling gspread-6.2.0:
      Successfully uninstalled gspread-6.2.0


In [2]:
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd
from google.colab import auth
from google.auth import default

# Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Sheet
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1wEvnusWWb7ztABVHk7I5u8F_uT3gSJaB3Lfcc0QYxLw/edit?gid=1787657154#gid=1787657154")

sheet1 = spreadsheet.worksheet("Form Responses 1")
sheet2 = spreadsheet.worksheet("Sheet2")

# Read data from Sheet1
df = get_as_dataframe(sheet1)  # remove empty rows

df.head()




,Timestamp,Email Address,Date of audit.,Reviewer Name,Ticket ID,Call recording if reviewing a call.,Emp. Name,Ticket View,Action in time,Courtesy,...,Smart Alternatives to Out of Stock Items,Use of Macros,Use of Mini Catalog Macros,Use of Tech,Final Rating For,Final Rating,Additional comments (if any),Need to add the feedback to scorecard?,Action by Agent—done?,Form Response Edit URL
0,4/1/2025 14:35:53,maya@clickforge.com,4/1/2025,Maya,1100494321,NaN,Elaunda Hood,level1,NaN,NaN,...,NaN,NaN,NaN,NaN,Use of Tech,Slight Opportunity,The order number in the subject or relevant to...,Yes,NaN,https://docs.google.com/forms/d/e/1FAIpQLScFPD...
1,4/1/2025 15:06:24,pratiksha@clickforge.com,4/1/2025,Pratiksha,1763986,NaN,Rahul Das,Internal expedite,NaN,NaN,...,NaN,NaN,NaN,NaN,Use of Tech,Meet,GOOD REVIEW\nCUSTOMERS COMMENT- Love it. \n\nC...,No,Done,https://docs.google.com/forms/d/e/1FAIpQLScFPD...
2,4/1/2025 15:22:50,pratiksha@clickforge.com,4/1/2025,Pratiksha,1759713,NaN,Jordyn Orchard,Jordyn Orchard,NaN,NaN,...,NaN,NaN,NaN,NaN,Promptness & Efficiency,Slight Opportunity,\nBAD REVIEW\nCUSTOMERS COMMENT- I shouldn't h...,Yes,Done,https://docs.google.com/forms/d/e/1FAIpQLScFPD...
3,4/1/2025 15:31:43,maya@clickforge.com,4/1/2025,Maya,1763460,NaN,Elaunda Hood,level1,NaN,NaN,...,NaN,NaN,NaN,NaN,Use of Tech,Strong Opportunity,No tracking added to order used for return,Yes,NaN,https://docs.google.com/forms/d/e/1FAIpQLScFPD...
4,4/1/2025 15:34:09,pratiksha@clickforge.com,4/1/2025,Pratiksha,1752376,NaN,Ashutosh Kumar Chandan,Ashutosh Kumar Chandan,NaN,NaN,...,NaN,NaN,NaN,NaN,Action in time,Meet,\nBAD REVIEW\nCUSTOMERS COMMENT- none\n\nCUSTO...,Yes,Done,https://docs.google.com/forms/d/e/1FAIpQLScFPD...


In [3]:
# Ensure Timestamp is in datetime format
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")

# Sort by Reviewer Name, then Timestamp
df = df.sort_values(by=["Reviewer Name", "Timestamp"], ascending=[True, True])

# Reset index if needed
df.reset_index(drop=True, inplace=True)

In [4]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"], errors="coerce")
df["Date of audit."] = pd.to_datetime(df["Date of audit."], errors="coerce").dt.date

# Sort before processing
df = df.sort_values(by=["Reviewer Name", "Date of audit.", "Timestamp"]).reset_index(drop=True)

from datetime import timedelta

# Initialize column
df["Time_Diff"] = pd.NaT

# Loop and apply condition
for i in range(len(df) - 1):
    curr = df.loc[i]
    next_ = df.loc[i + 1]

    if (
        curr["Timestamp"].date() == curr["Date of audit."]
        and next_["Date of audit."] == curr["Date of audit."]
        and next_["Reviewer Name"] == curr["Reviewer Name"]
    ):
        # Calculate time difference: next row timestamp - current row timestamp
        time_diff = next_["Timestamp"] - curr["Timestamp"]
        df.loc[i, "Time_Diff"] = time_diff

df["Time_Diff_Minutes"] = df["Time_Diff"].apply(
    lambda x: x.total_seconds() / 60 if pd.notnull(x) else None
)

<ipython-input-4-78068680e4c1>:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0 days 06:00:04' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.loc[i, "Time_Diff"] = time_diff


In [5]:
import pandas as pd

# 30-minute threshold
threshold = pd.Timedelta(minutes=30)

# Step 1: Only use values < 30 mins to compute reviewer-wise averages
valid_df = df[df["Time_Diff"] < threshold].dropna(subset=["Time_Diff"])
avg_lookup = valid_df.groupby("Reviewer Name")["Time_Diff"].mean().to_dict()

# Step 2: Logic to apply like Excel IF + AVERAGEIFS
def apply_excel_logic(row):
    td = row["Time_Diff"]
    reviewer = row["Reviewer Name"]
    avg = avg_lookup.get(reviewer)

    if pd.notnull(td) and td > threshold:
        return avg if pd.notnull(avg) else pd.NaT  # leave blank if no valid avg
    return td  # keep original if ≤ 30 mins

# Step 3: Apply the logic
df["Final_Time_Diff"] = df.apply(apply_excel_logic, axis=1)

# Step 4: Format to HH:MM:SS
df["Final_Time_Diff_HHMMSS"] = df["Final_Time_Diff"].apply(
    lambda x: str(x).split(" ")[-1] if pd.notnull(x) else None
)

In [6]:
# Install dependencies (only needed once in Colab)
!pip install pydub
!apt install ffmpeg -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [7]:
# ✅ All necessary imports
import requests
from pydub import AudioSegment
from io import BytesIO
import pandas as pd  # Already present if you're using df


In [8]:
# ✅ Clean and consistent duration extractor
def get_audio_duration_from_gdrive(url):
    try:
        if "drive.google.com" not in url:
            return None

        # Extract file ID
        if "file/d/" in url:
            file_id = url.split("file/d/")[1].split("/")[0]
        elif "id=" in url:
            file_id = url.split("id=")[1].split("&")[0]
        else:
            return None

        # Construct direct download URL
        download_url = f"https://drive.google.com/uc?export=download&id={file_id}"
        response = requests.get(download_url)

        # Convert to audio segment
        try:
            audio = AudioSegment.from_file(BytesIO(response.content), format="mp3")
        except:
            audio = AudioSegment.from_file(BytesIO(response.content))

        duration = len(audio) / 1000  # in seconds
        return float(round(duration, 2))
    except:
        return None

In [9]:
df['Ticket ID Audio Duration (sec)'] = df['Ticket ID'].apply(
    lambda x: get_audio_duration_from_gdrive(str(x)) if pd.notna(x) else None)

df['Call Recording Duration (sec)'] = df['Call recording if reviewing a call.'].apply(
    lambda x: get_audio_duration_from_gdrive(str(x)) if pd.notna(x) else None)


In [10]:
# Clear and update Sheet2
sheet2.clear()
set_with_dataframe(sheet2, df)

In [11]:
# Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Open Sheet
spreadsheet1 = gc.open_by_url("https://docs.google.com/spreadsheets/d/1Ih3q9x_16IKD_NEaiX7RqK0fHKH1HYoARKfYJcieP6Y/edit?gid=0#gid=0")

sheet1 = spreadsheet1.worksheet("Sheet1")

sheet1.clear()
set_with_dataframe(sheet1, df)

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
df.to_excel('/content/drive/MyDrive/Drive/Colab Notebooks/Duration3.xlsx', index=False)